In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

In [3]:
# Define constants
g = 1.0           # Atom-cavity coupling strength
kappa = 1.0       # Cavity decay rate
gamma = 0.5       # Atomic spontaneous emission rate
T = 10            # Pulse duration
n_cavity = 2      # Photon number states: 0 or 1
n_atom = 3        # Atom: |0>, |1>, |e>

# Basis states
# Atom: |0>, |1>, |e> → index 0, 1, 2
# Cavity: |0>, |1> → index 0, 1

# Identity operators
I_atom = qeye(n_atom)
I_cavity = qeye(n_cavity)

# Atom operators
proj_1 = basis(n_atom, 1) * basis(n_atom, 1).dag()
proj_e = basis(n_atom, 2) * basis(n_atom, 2).dag()
sig_1e = basis(n_atom, 1) * basis(n_atom, 2).dag()  # |1⟩⟨e|
sig_e1 = sig_1e.dag()                               # |e⟩⟨1|

# Cavity operators
a = destroy(n_cavity)   # Cavity annihilation
adag = a.dag()

# Tensor operators (system = atom ⊗ cavity)
H_int = g * tensor(sig_e1, a) + g * tensor(sig_1e, adag)  # Jaynes-Cummings

# Collapse operators
c_ops = [
    np.sqrt(kappa) * tensor(I_atom, a),             # Cavity decay
    np.sqrt(gamma) * tensor(basis(n_atom, 0) * basis(n_atom, 2).dag(), I_cavity),  # |e> → |0>
    np.sqrt(gamma) * tensor(basis(n_atom, 1) * basis(n_atom, 2).dag(), I_cavity),  # |e> → |1>
]

In [4]:
# Atom initial state: (|0> + |1>) / sqrt(2)
psi_atom = (basis(n_atom, 0) + basis(n_atom, 1)).unit()

# Cavity initial state: 1 photon (|1>)
psi_cavity = basis(n_cavity, 1)

# Full initial state
psi0 = tensor(psi_atom, psi_cavity)

In [5]:
# Time list
tlist = np.linspace(0, T, 500)

# Evolve
result = mesolve(H_int, psi0, tlist, c_ops, [])

/home/lz/Documents/uni/master/master_thesis/code/.venv/lib/python3.13/site-packages/qutip/solver/solver_base.py:576: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


In [6]:
result.expect()

TypeError: 'list' object is not callable